# Cardiac Cine MRI with Arrhythmias

In [ ]:
using KomaMRI, PlotlyJS, Plots, Printf, Suppressor; #hide
include(joinpath(dirname(pathof(KomaMRI)), "../examples/3.tutorials/utils/RRVariability.jl")); #hide
sys = Scanner(); #hide

This tutorial shows how to simulate cardiac cine MRI using Koma,
including cases with variable RR intervals (i.e., arrhythmias). You'll learn how to:

1. Simulate a clean cine acquisition with constant RR intervals.
2. Introduce arrhythmias (variable RR intervals) into the cardiac phantom.
3. Observe how this desynchronization degrades image quality.
4. Correct the acquisition by synchronizing the sequence with the phantom’s RR variability (manual triggering).

### 1. Constant RR for Phantom and Sequence

We will begin by simulating a cardiac cine on a myocardial phantom with a constant RR interval.
We'll use the `heart_phantom` function to create a ring-shaped phantom filled with blood, resembling the left ventricle:

In [ ]:
obj = heart_phantom();

By default, this phantom exhibits periodic contraction and rotation with a 1-second period:

In [ ]:
p1 = plot_phantom_map(obj, :T1 ; height=450, time_samples=21) #hide

As shown in previous tutorials, the phantom's motion is defined by its `motion` field.
Until now, this motion has typically consisted of a single `Motion` component.
In this case, however, it consists of two independent motions: a contraction (`HeartBeat`)
and a `Rotation`. These two are grouped together in a `MotionList` structure:

In [ ]:
obj.motion

Now, we will create a bSSFP cine sequence with the following parameters:

In [ ]:
RRs          = [1.0]       # [s] constant RR interval
N_matrix     = 50          # image size = N x N
N_phases     = 25          # Number of cardiac phases
FOV          = 0.11        # [m]
TR           = 25e-3       # [s]
flip_angle   = 10          # [º]
adc_duration = 0.2e-3;     # [s]

In [ ]:
seq = bSSFP_cine(
    FOV, N_matrix, TR, flip_angle, RRs, N_phases, sys;
    N_dummy_cycles = 40, adc_duration = adc_duration,
);

# Simulation  #hide
raw1 = @suppress simulate(obj, seq, sys); #hide
# Reconstruction #hide
frames1 = @suppress reconstruct_cine(raw1, seq, N_matrix, N_phases); #hide

The simulation and subsequent reconstruction produces the following cine frames,
which look clean and temporally coherent:

In [ ]:
fps = 25 #hide
p2 = @suppress plot_cine(frames1, fps; Δt=TR, filename="../assets/tut-7-frames1.gif"); #hide
display(p2);

### 2. Arrhythmic Phantom: Variable RR, Constant Sequence

Now, we will introduce arrhythmias into the phantom by varying its RR intervals.
However, the sequence will still assume a constant RR interval of 1 second.

In [ ]:
RRs = [900, 1100, 1000, 1000, 1000, 800] .* 1e-3;

Let's apply the new `RRs` to the phantom:

In [ ]:
# Take the time curve from the contraction motion:
t_curve = obj.motion.motions[1].time
# Generate a new time curve:
t_curve_new = TimeCurve(
    t = t_curve.t,
    t_unit = t_curve.t_unit,
    periodic = true,
    periods = RRs
)
# Assign the new time curve to both the contraction and the rotation:
obj.motion.motions[1].time = t_curve_new
obj.motion.motions[2].time = t_curve_new;

Let’s visualize how the motion pattern has changed, now with variable-duration RR intervals:

In [ ]:
p3 = plot_phantom_map(obj, :T1 ; height=450, time_samples=41) #hide

Since the sequence still assumes a constant RR interval, it becomes unsynchronized with the phantom.
This results in artifacts and temporal inconsistencies in the cine images. We will showcase these images in the next section.

In [ ]:
# Simulation  #hide
raw2 = @suppress simulate(obj, seq, sys) #hide
# Reconstruction #hide
frames2 = @suppress reconstruct_cine(raw2, seq, N_matrix, N_phases); #hide

plot_cine(frames2, fps; Δt=TR, filename="tut-7-frames2.gif");

### 3. Prospective Triggering: Resynchronized Acquisition
To correct this, we synchronize the sequence **manually** by providing it the same RR intervals as the phantom:

In [ ]:
seq = bSSFP_cine(
    FOV, N_matrix, TR, flip_angle, RRs, N_phases, sys;
    N_dummy_cycles = 40, adc_duration = adc_duration,
);

This approach manually mimics cardiac triggering. The resulting cine is
once again correctly aligned, despite the underlying arrhythmia.

In the future, this synchronization will be handled automatically
through upcoming support for trigger extensions in the sequence framework.

In [ ]:
# Simulation  #hide
raw3 = @suppress simulate(obj, seq, sys) #hide
# Reconstruction #hide
frames3 = @suppress reconstruct_cine(raw3, seq, N_matrix, N_phases); #hide

plot_cine(frames3, fps; Δt=TR, filename="tut-7-frames3.gif");

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*